# ML: Assignment 3

## Imports

In [1]:
from src.data_loading import DataLoader
from src.data_processing import Process

from src.next_word_model import NextWordModel

from src.prediction_evaluation import Evaluator

## Pipeline

### Data Loading & Processing

In [2]:
dl = DataLoader()
process = Process(dl)

In [3]:
X_train, X_test, y_train, y_test = process.process(force=False)


[Errno 2] No such file or directory: '/Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/_objects/data.pkl'
No preprocessed file to load - <force> will be ignored.


/Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/data_processing.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["hasLink"] = df.text.str.contains(url_regex)
/Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/data_processing.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["hasLink"] = df.text.str.contains(url_regex)


In [4]:
print(len(X_train))

38132


### Model Init & Training

In [5]:
model = NextWordModel(
    sequence_length = process.sequence_length,
    vocab_size = process.vocab_size,
    load_existing = True
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             52330     
                                                                 
 lstm (LSTM)                 (None, 1, 100)            44400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 5233)              528533    
                                                                 
Total params: 715,763
Trainable params: 715,763
Non-trainable params: 0
_________________________________________________________________
None
Loading existing model failed:
module 'tensor

In [6]:
model.train(X_train, y_train)

Epoch 1/100


2023-02-01 17:38:42.325944: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


298/298 [==============================] - 4s 7ms/step - loss: 6.8873 - accuracy: 0.0550
Epoch 2/100
298/298 [==============================] - 2s 7ms/step - loss: 6.3523 - accuracy: 0.0554
Epoch 3/100
298/298 [==============================] - 2s 7ms/step - loss: 6.3085 - accuracy: 0.0554
Epoch 4/100
298/298 [==============================] - 2s 7ms/step - loss: 6.2842 - accuracy: 0.0564
Epoch 5/100
298/298 [==============================] - 2s 7ms/step - loss: 6.2585 - accuracy: 0.0621
Epoch 6/100
298/298 [==============================] - 2s 8ms/step - loss: 6.1845 - accuracy: 0.0661
Epoch 7/100
298/298 [==============================] - 2s 8ms/step - loss: 6.1023 - accuracy: 0.0696
Epoch 8/100
298/298 [==============================] - 3s 8ms/step - loss: 6.0390 - accuracy: 0.0718
Epoch 9/100
298/298 [==============================] - 3s 9ms/step - loss: 5.9795 - accuracy: 0.0732
Epoch 10/100
298/298 [==============================] - 2s 8ms/step - loss: 5.9204 - accuracy: 0.0790
E

INFO:tensorflow:Assets written to: /Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/_objects/lstm.nextword.model/assets


INFO:tensorflow:Assets written to: /Users/tilmankerl/Documents/UNI/01_WS22/Machine Learning/exercises/03/src/_objects/lstm.nextword.model/assets


### Model Evaluation

In [7]:
ev = Evaluator(model)

In [8]:
ev.evaluate_lemma(X_test, y_test)

Pair(true=39, pred=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32))
39
